# Make a .gif of optical imagery

In [ ]:
import ee
import matplotlib.pyplot as plt
import numpy as pd
import geopandas as gpd
import os
import shapely
from shapely.geometry import Polygon
import geedim as gd
import PIL
import io
import requests
import datetime
from tqdm.auto import tqdm

In [ ]:
# -----Initialize GEE
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
# -----Define paths in directory
# path to data
data_path = '/Users/raineyaberle/Research/PhD/Hubbard/'
# output path for figures 
out_path = os.path.join(data_path, 'optical_imagery', 'frames')

# -----Define search and plotting settings
# Target CRS for output figures
crs = 'EPSG:3338'
# Start and end dates
date_start = '2013-04-01'
date_end = '2024-02-28'

In [ ]:
# -----Load AOI for image querying
# Load clipped glacier outline
aoi_fn = os.path.join(data_path, 'RGI', 'Hubbard_boundaries_clipped.shp')
aoi = gpd.read_file(aoi_fn)
# Define bounding box
xmin, xmax = 7.9e5, 8.15e5
ymin, ymax = 1.195e6, 1.220e6
bbox = Polygon([[xmin, ymin],
                [xmax, ymin],
                [xmax, ymax],
                [xmin, ymax],
                [xmin, ymin]])
bbox_df = gpd.GeoDataFrame(geometry=[bbox], crs=aoi.crs)

# -----Plot
fig, ax = plt.subplots()
aoi.plot(ax=ax)
bbox_df.plot(ax=ax, facecolor='None', edgecolor='m')
plt.show()

In [ ]:
def plot_save_image_thumbnail(image, bounds, image_date, dataset, figure_fn, out_path):
    # Create out_path if it doesn't exist
    if not os.path.exists(out_path):
        os.mkdir(out_path)
        print('Created out_path:', out_path)
        
    # Plot
    plt.rcParams.update({'font.size': 12, 'font.sans-serif': 'Arial'})
    fig, ax = plt.subplots(figsize=(8,8))
    ax.imshow(image, extent=(bounds[0], bounds[2], bounds[1], bounds[3]))
    ax.set_title(image_date + '\n' + dataset)
    plt.close()
        
    # Save figure to file
    fig.savefig(figure_fn, dpi=250, bbox_inches='tight')
    print('Figure saved to file:', figure_fn)

    return 

def apply_Landsat_scale_factors(image_ee):
    opticalBands = image_ee.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image_ee.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image_ee.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

def construct_image_thumbnail_figures(date_start, date_end, dataset, aoi, crs, out_path):
    """
    Parameters
    ----------
    date_start: str
        start date for image querying (e.g, "2013-01-01")
    date_end: str
        end date for image querying (e.r., "2013-02-01")
    dataset: str
        which dataset / image collection to query ("Landsat", "Sentinel-2_SR", "Sentinel-2_TOA")
    aoi: geopandas.geodataframe.GeoDataFrame
        area of interest for querying and clipping imagery
    crs: str
        target Coordinate Reference System (CRS) for imagery (e.g., "EPSG:4326")

    Returns
    -------
    None
    """

    # -----Reformat AOI for querying 
    # Reproject to WGS84
    aoi_wgs = aoi.to_crs('EPSG:4326')
    # Grab AOI bounding box
    xmin, xmax = aoi_wgs.geometry[0].bounds[0], aoi_wgs.geometry[0].bounds[2]
    ymin, ymax = aoi_wgs.geometry[0].bounds[1], aoi_wgs.geometry[0].bounds[3]
    aoi_bb = Polygon([[xmin, ymin], [xmax, ymin], [xmax, ymax], [xmin, ymax], [xmin, ymin]])
    # Reformat to geojson format for querying gee
    aoi_ee = {'type': 'Polygon',
              'coordinates': [[[xmin, ymin], 
                               [xmax, ymin], 
                               [xmax, ymax], 
                               [xmin, ymax], 
                               [xmin, ymin]
                          ]] }

    # -----Query GEE for imagery (via geedim)
    if dataset == 'Landsat':
        # Landsat 8
        im_col_gd_8 = gd.MaskedCollection.from_name('LANDSAT/LC08/C02/T1_L2').search(start_date=date_start,
                                                                                     end_date=date_end,
                                                                                     mask=True,
                                                                                     region=aoi_ee,
                                                                                     fill_portion=70)
        # Landsat 9
        im_col_gd_9 = gd.MaskedCollection.from_name('LANDSAT/LC09/C02/T1_L2').search(start_date=date_start,
                                                                                     end_date=date_end,
                                                                                     mask=True,
                                                                                     region=aoi_ee,
                                                                                     fill_portion=70)
        # Merge Landsat 8 and 9 collections
        im_col_ee = im_col_gd_8.ee_collection.merge(im_col_gd_9.ee_collection)
        # define how to display image
        visualization = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 1.0, 'dimensions': 768,
                         'region': aoi_ee}
        scale = 30
        
    elif dataset == 'Sentinel-2_TOA':
        im_col_gd = gd.MaskedCollection.from_name('COPERNICUS/S2_HARMONIZED').search(start_date=date_start,
                                                                                     end_date=date_end,
                                                                                     mask=True,
                                                                                     region=aoi_ee,
                                                                                     fill_portion=70)
        im_col_ee = im_col_gd.ee_collection
        # define how to display image
        visualization = {'bands': ['B4', 'B3', 'B2'], 'min': 0.0, 'max': 1e4, 'dimensions': 768,
                         'region': aoi_ee}
        scale = 30
    elif dataset == 'Sentinel-2_SR':
        im_col_gd = gd.MaskedCollection.from_name('COPERNICUS/S2_SR_HARMONIZED').search(start_date=date_start,
                                                                                        end_date=date_end,
                                                                                        mask=True,
                                                                                        region=aoi_ee,
                                                                                        fill_portion=70)
        im_col_ee = im_col_gd.ee_collection
        # define how to display image
        visualization = {'bands': ['B4', 'B3', 'B2'], 'min': 0.0, 'max': 1e4, 'dimensions': 768,
                         'region': aoi_ee}
        scale = 30
    else:
        print("'dataset' variable not recognized. Please set to 'Landsat', 'Sentinel-2_TOA', or 'Sentinel-2_SR'. Exiting...")
        return 'N/A'

    # -----Print size of image collection
    im_col_ee_size = im_col_ee.size().getInfo()
    print(f'Number of images in collection = {im_col_ee_size}')
    if im_col_ee_size < 1:
        print('Exiting...')
        return

    # -----Iterate over image IDs
    # Grab image IDs
    im_ids = im_col_ee.aggregate_array('system:id').getInfo()
    # Iterate
    for im_id in tqdm(im_ids):
        # Create image
        im_ee = gd.MaskedImage.from_id(im_id, region=aoi_ee, mask=False).ee_image
        # Grab image date
        im_date = ee.Date(im_ee.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        # Define figure name
        fig_fn = os.path.join(out_path, str(im_date) + '_' + str(dataset) + '.png')
        # Check if figure already exists in directory before continuing
        if os.path.exists(fig_fn):
            continue
        # Request image thumbnail
        # Apply scale factors for Landsat
        if dataset=='Landsat':
            im_ee = apply_Landsat_scale_factors(im_ee)
        
        # Reproject the Earth Engine image to CRS
        im_ee_reproj = im_ee.reproject(crs, scale=scale)
        # Reproject AOI, grab image bounds
        aoi_reproj = aoi.to_crs(crs)
        bounds = aoi_reproj.geometry[0].bounds
        # Fetch the image URL from Google Earth Engine
        im_url = im_ee_reproj.clip(aoi_ee).getThumbURL(visualization)
        # Fetch the image and convert it to a PIL Image object
        response = requests.get(im_url)
        image_bytes = io.BytesIO(response.content)
        image = PIL.Image.open(image_bytes)

        # Plot and save figure
        plot_save_image_thumbnail(image, bounds, im_date, dataset, fig_fn, out_path)

    return

In [ ]:
# -----Landsat
dataset = 'Landsat'
construct_image_thumbnail_figures(date_start, date_end, dataset, bbox_df, crs, out_path)

In [ ]:
# -----Sentinel-2 SR
date_start = '2019-01-01'
date_end = '2020-01-01'
dataset = 'Sentinel-2_SR'
construct_image_thumbnail_figures(date_start, date_end, dataset, bbox_df, crs, out_path)

In [ ]:
# -----Landsat
date_start = '2016-01-01'
date_end = '2019-01-01'
dataset = 'Sentinel-2_TOA'
construct_image_thumbnail_figures(date_start, date_end, dataset, bbox_df, crs, out_path)